# Calibrate model to market implied volatility

In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd

import sys, os
import yfinance as yf


from datetime import datetime, timezone

In [ ]:

sys.path.append(os.path.abspath(".."))


from pricers.monte_carlo_pricer import mc_pricer
from models.local_vol import build_dupire_local_vol_surface

## A. Local Volatility Model

Vol is a function of price & time:

$$\sigma = \sigma(S,t)$$

MC simulation becomes:

$$dS = \mu S\,dt + \sigma(S,t)S\,dW$$

You need **local vol surface** (Dupire Formula), calibrated from the market.

We look at how option prices change with:

- **maturity** → $\partial C / \partial T$
- **strike** → $\partial^2 C / \partial K^2$

Those sensitivities contain information about volatility.

Dupire says:

$$\sigma^2_{\text{local}}(K,T) = \frac{\frac{\partial C}{\partial T}}{\frac{1}{2}K^2\frac{\partial^2 C}{\partial K^2}}$$




In [ ]:

ticker = yf.Ticker("NVDA")
expiry = ticker.options[1]             # one expiry for now
chain = ticker.option_chain(expiry)
spot = ticker.info['currentPrice'] 

calls = chain.calls

# we could filter bid/ask zero contracts
# calls = calls[(calls["bid"] > 0) & (calls["ask"] > 0)]     

# we could also restrict strikes near ATM (far strikes are messy)
# calls = calls[(calls["strike"] > spot)]    #strike above spot

calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,NVDA260116C00000500,2026-01-02 17:38:54+00:00,0.5,188.20,187.90,189.00,2.050003,1.101264,31.0,48749.0,15.562500,True,REGULAR,USD
1,NVDA260116C00001000,2026-01-02 17:42:10+00:00,1.0,188.21,187.15,188.45,1.480011,0.792594,67.0,18022.0,20.125004,True,REGULAR,USD
2,NVDA260116C00001500,2026-01-02 19:00:09+00:00,1.5,186.99,186.65,187.75,-0.429993,-0.229427,20.0,2099.0,14.921876,True,REGULAR,USD
3,NVDA260116C00002000,2026-01-02 19:00:09+00:00,2.0,186.53,186.15,187.25,-0.290009,-0.155234,20.0,19925.0,13.500002,True,REGULAR,USD
4,NVDA260116C00002500,2025-12-30 14:56:24+00:00,2.5,186.29,185.70,186.90,0.000000,0.000000,61.0,1027.0,13.437502,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,NVDA260116C02280000,2024-06-07 19:35:08+00:00,2280.0,91.40,90.35,92.25,-0.599998,-0.652172,5.0,45.0,13.694948,False,REGULAR,USD
457,NVDA260116C02300000,2024-06-07 19:48:58+00:00,2300.0,90.55,88.60,90.20,90.550000,NaN,5.0,0.0,13.572267,False,REGULAR,USD
458,NVDA260116C02400000,2024-06-07 17:28:14+00:00,2400.0,81.49,79.55,81.20,81.490000,NaN,3.0,15.0,13.003908,False,REGULAR,USD
459,NVDA260116C02450000,2024-06-07 19:23:41+00:00,2450.0,75.88,75.45,77.05,75.880000,NaN,3.0,NaN,12.751589,False,REGULAR,USD


In [4]:
expiry

'2026-01-16'

#### some comments:
- Many strikes/maturities have no liquidity 
- Some IVs are near 0 or extremely high



In [6]:
spot

188.85

In [7]:
ticker.options

('2026-01-09',
 '2026-01-16',
 '2026-01-23',
 '2026-01-30',
 '2026-02-06',
 '2026-02-20',
 '2026-03-20',
 '2026-04-17',
 '2026-05-15',
 '2026-06-18',
 '2026-07-17',
 '2026-08-21',
 '2026-09-18',
 '2026-12-18',
 '2027-01-15',
 '2027-06-17',
 '2027-09-17',
 '2027-12-17',
 '2028-01-21')

In [8]:
expiries = list(ticker.options)                       # ['2026-01-09', ...]
exp_dt = pd.to_datetime(expiries)

val_dt = pd.Timestamp.today().normalize()             # or pd.Timestamp("2026-01-04")

ttm_years = ((exp_dt - val_dt) / pd.Timedelta(days=365)).to_numpy()
expiry_to_ttm = dict(zip(expiries, ttm_years))  

In [9]:
np.array(ttm_years)

array([0.01369863, 0.03287671, 0.05205479, 0.07123288, 0.09041096,
       0.12876712, 0.20547945, 0.28219178, 0.35890411, 0.45205479,
       0.53150685, 0.62739726, 0.70410959, 0.95342466, 1.03013699,
       1.44931507, 1.70136986, 1.95068493, 2.04657534])

In [10]:
# plot a market surface:

market_surface = {}

# maturities = np.array(ticker.options)
# for expiry in maturities:
#     chain = ticker.option_chain(expiry)
#     strikes = chain.calls["strike"]
#     IV = chain.calls["impliedVolatility"]
#     market_surface[expiry] = dict(zip(strikes, IV))


for expiry_str, ttm in expiry_to_ttm.items():
    chain = ticker.option_chain(expiry_str)         
    strikes = chain.calls["strike"].to_numpy()
    iv = chain.calls["impliedVolatility"].to_numpy()

    market_surface[ttm] = dict(zip(strikes, iv)) 
    

In [11]:


# strikes = sorted(list(set(k for v in market_surface.values() for k in v.keys())))
# maturities = list(market_surface.keys())

# Z = np.zeros((len(maturities), len(strikes)))
# for i,m in enumerate(maturities):
#     for j,s in enumerate(strikes):
#         Z[i,j] = market_surface[m].get(s, np.nan)

# X, Y = np.meshgrid(strikes, maturities)

# fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z)])
# fig.update_layout(title="Volatility Surface",
#                   scene=dict(xaxis_title="Strike",
#                              yaxis_title="Maturity",
#                              zaxis_title="Vol"))
# fig.show()


In [12]:
# #Trim to minimum common strikes across maturities
# min_len = min(len(v) for v in market_surface.values())

# strikes_common = sorted(list(set(k for v in market_surface.values() for k in v.keys())))
# strikes_common = strikes_common[:min_len]   # first min_len 

# maturities = list(market_surface.keys())

# Z = np.zeros((len(maturities), min_len))
# for i,m in enumerate(maturities):
#     sorted_strikes = sorted(market_surface[m].keys())[:min_len]
#     for j,s in enumerate(sorted_strikes):
#         Z[i,j] = market_surface[m][s]

# X, Y = np.meshgrid(strikes_common, maturities)

# fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z)])
# fig.update_layout(scene=dict(
#     xaxis_title="Strike",
#     yaxis_title="Maturity",
#     zaxis_title="Vol"))
# fig.show()


In [13]:
#### Comments:
# mixing all strikes without aligning them properly ( same lenght but not same set of Ks)
common_strikes = set.intersection(*(set(d.keys()) for d in market_surface.values())) #built-in to return elements common to all sets
common_strikes = sorted(common_strikes)

maturities = list(market_surface.keys())  # expiry axis

# build iv surface:
Z = np.zeros((len(maturities), len(common_strikes)))

for i, m in enumerate(maturities):
    for j, s in enumerate(common_strikes):
        Z[i, j] = market_surface[m][s]



X, Y = np.meshgrid(common_strikes, maturities)

fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z)])
fig.update_layout(
    title="Volatility Surface (Common Strikes Aligned)",
    scene=dict(
        xaxis_title="Strike",
        yaxis_title="Maturity",
        zaxis_title="Implied Volatility"
    )
)
fig.show()


In [14]:
# test case since i get nan value from yahoo market surface..
K = np.array([80, 90, 100, 110, 120])
T = np.array([0.25, 0.5, 1.0, 2.0])

# Black-Scholes style smile
def smile(k): return 0.18 + 0.0005*(k-100)**2/5

IV_grid = np.array([
    [smile(k)     for k in K],       # short-dated, mild smile
    [smile(k)*1.05 for k in K],      # steeper with maturity
    [smile(k)*1.10 for k in K],
    [smile(k)*1.15 for k in K],
])

market_surface = {
    "T0.25": dict(zip(K, IV_grid[0])),
    "T0.50": dict(zip(K, IV_grid[1])),
    "T1.00": dict(zip(K, IV_grid[2])),
    "T2.00": dict(zip(K, IV_grid[3])),
}



In [15]:
KK, TT = np.meshgrid(K, T)

fig = go.Figure(data=[go.Surface(
    x=KK,   # Strike axis
    y=TT,   # Maturity axis
    z=IV_grid, # Vol matrix
    colorbar=dict(title="Vol")
)])

fig.update_layout(
    title="Implied Volatility Surface",
    scene=dict(
        xaxis_title="Strike",
        yaxis_title="Maturity",
        zaxis_title="IV"
    )
)
fig.show()


In [ ]:


LV = build_dupire_local_vol_surface(100, market_surface, r=0.03)

print("Local Vol at ATM:", LV(1.0, 100)[0,0])


IV_grid:
 [[0.22   0.19   0.18   0.19   0.22  ]
 [0.231  0.1995 0.189  0.1995 0.231 ]
 [0.242  0.209  0.198  0.209  0.242 ]
 [0.253  0.2185 0.207  0.2185 0.253 ]]
0.21999999999999992
0.1899999999999999
0.17999999999999988
0.18999999999999992
0.2199999999999999
0.23100000000000004
0.19949999999999998
0.18899999999999997
0.19949999999999998
0.231
0.2420000000000001
0.20899999999999996
0.19799999999999998
0.20900000000000005
0.24200000000000005
0.2530000000000001
0.21849999999999997
0.207
0.21849999999999997
0.253
Local Vol at ATM: 0.3176966451246616


#### Local Vol MC Price: 

In [17]:

def european_call_payoff(path, K):
    return np.maximum(path[-1] - K, 0.0)



price, ci, stderr, beta = mc_pricer(
    payoff_fn=european_call_payoff,
    payoff_args=[K],
    S0=100, r=0.03, sigma=None, # sigma ignored!
    T=1.0, N=250, M=20000,
    sim_method="local_vol",  
    LV=LV,   
    seed=42
)


In [18]:
print("Local Vol MC Price =", price)
print("95% CI =", ci)
print("std error =", stderr)

Local Vol MC Price = 12.2159481233037
95% CI = (11.662425271528742, 12.76947097507866)
std error = 0.28241480769089455
